In [1]:
from mcr.gate_apply import PauliBit
from mcr.mcr_optimize import find_mcr
from mcr.gate_apply import grouping, loop_optimization
from mcr.equiv_check import pauli_bit_equivalence_check, equivalence_check_via_mqt_qcec
from qulacs import QuantumCircuit
from mcr.gate_apply import set_clifford_to_qulacs
from mcr.mcr_optimize import find_nontrivial_swap

In [4]:
import numpy as np
import pickle
filetype = "seq"  # "small" or "seq"
# with open(f"unopt_{filetype}.pickle", "rb") as f:
with open(f"unopt_3.pickle", "rb") as f:
    seq = pickle.load(f)
data = []
for elem in seq:
    sgn = str(elem[1])[0]
    pauli_str = str(elem[1])[1:]
    if sgn == "+":
        data.append(PauliBit(pauli_str, np.pi/4))
    else:
        assert sgn == "-", f"Unexpected sign: {sgn}"
        data.append(PauliBit(pauli_str, -np.pi/4))

In [3]:
import pickle
def load_data_from_pickle(filepath: str, nqubits: int) -> list:
    """pickle形式のファイルを読み込む

    Args:
        filepath (str): pickle形式のファイルパス

    Returns:
        list: 読み込んだデータのリスト
    """
    with open(filepath, mode="br") as f:
        data = pickle.load(f)
    pauli_bit_sequences = []
    for elem in data:
        initial_pauli_str, theta, qubit_indices = elem
        pauli_str = ["I"] * nqubits
        for pauli, qubit_idx in zip(initial_pauli_str, qubit_indices):
            pauli_str[qubit_idx] = pauli
        pauli_bit = PauliBit(pauli_str, theta)
        pauli_bit_sequences.append(pauli_bit)
    return pauli_bit_sequences
nqubits = 6
data = load_data_from_pickle("te_pai_data.pickle", nqubits=nqubits)

In [4]:
_, non_clifford_rots = loop_optimization(data)
groups = grouping(non_clifford_rots)

3-iteration optimization applied!
optimization result: 2341 -> 1912


In [5]:
# groups

In [6]:
# 何でもかんでも入れ替えれば良いという訳でもなさそう
# Q. 入れ替えられる候補がN通りあったとして、2^N通りのパターンを調べ、一番良いものを選ぶというような手法を取ることはできる？
def mcr_swap(pauli_bit_groups):
    removed_group_indices = set()
    counter = 0
    for i in range(len(pauli_bit_groups) - 1):
        if i not in removed_group_indices:
            left_data = pauli_bit_groups[i]
            right_data = pauli_bit_groups[i + 1]
            sols = find_mcr(left_data, right_data)
            if sols:
                for gates in sols:
                    gate_a, gate_b, gate_c, gate_d = [gate[1:] for gate in gates]
                    print(f"MCR swap!: {i}, {i + 1} -> {gate_a}, {gate_b}, {gate_c}, {gate_d}")
                    print(f'left_data: {left_data}')
                    print(f'right_data: {right_data}')
                    left_data_for_swap, reduced_left_data = [], []
                    for ele in left_data:
                        if ele.get_pauli_str() in [gate_a, gate_b]:
                            left_data_for_swap.append(ele)
                        else:
                            reduced_left_data.append(ele)
                    assert len(left_data_for_swap) == 2, (
                        f"Expected 2 elements for swap, got {len(left_data_for_swap)}"
                    )

                    right_data_for_swap, reduced_right_data = [], []
                    for ele in right_data:
                        if ele.get_pauli_str() in [gate_c, gate_d]:
                            right_data_for_swap.append(ele)
                        else:
                            reduced_right_data.append(ele)

                    assert len(right_data_for_swap) == 2, (
                        f"Expected 2 elements for swap, got {len(right_data_for_swap)}"
                    )
                    pauli_bit_groups[i] = reduced_left_data + right_data_for_swap
                    pauli_bit_groups[i + 1] = left_data_for_swap + reduced_right_data
                    removed_group_indices.add(i + 1)
                    # removed_group_indices.add(i + 2)
                    counter += 1
                    break
    new_data = [ele for group in pauli_bit_groups for ele in group]
    # print(f"Total MCR swaps made: {counter}")
    return new_data

In [7]:
from mcr.gate_apply import synthesize_sequence
def test_algorithm(pauli_bit_lst):
    clifford_lst = []
    clifford, data_for_optimization = loop_optimization(pauli_bit_lst)
    clifford_lst.extend(clifford)
    flag = True
    length = len(data_for_optimization)
    while flag:
        groups = grouping(data_for_optimization)
        new_data = mcr_swap(groups)
        clifford, data_for_optimization = loop_optimization(new_data)
        if len(data_for_optimization) >= length:
            flag = False
            clifford_lst.extend(clifford)
        else:
            print(f"🎉 Successful optimization using MCR! {length} -> {len(data_for_optimization)}")
            length = len(data_for_optimization)
            clifford_lst.extend(clifford)
    return clifford_lst, data_for_optimization
clifford_lst, optimized_data = test_algorithm(data)

3-iteration optimization applied!
optimization result: 2341 -> 1912
MCR swap!: 70, 71 -> XIIIIX, YIIIIY, IIIIIZ, ZIIIII
left_data: [PauliBit(-0.02454369260617026*IIIXXI), PauliBit(-0.02454369260617026*IYYIII), PauliBit(-0.02454369260617026*XIIIIX), PauliBit(-0.02454369260617026*YIIIIY), PauliBit(-0.02454369260617026*ZIIIIZ)]
right_data: [PauliBit(-0.02454369260617026*IIIIIZ), PauliBit(-0.02454369260617026*IIIIZZ), PauliBit(-0.02454369260617026*IIIZII), PauliBit(-0.04908738521234052*ZIIIII), PauliBit(0.02454369260617026*ZZIIII)]
MCR swap!: 204, 205 -> IIIIIZ, ZIIIII, XIIIIX, YIIIIY
left_data: [PauliBit(-0.02454369260617026*IIIIIZ), PauliBit(-0.04908738521234052*IIIZII), PauliBit(-0.02454369260617026*IXXIII), PauliBit(-0.04908738521234052*IZZIII), PauliBit(-0.02454369260617026*ZIIIII)]
right_data: [PauliBit(-0.02454369260617026*IIXXII), PauliBit(-0.04908738521234052*XIIIIX), PauliBit(-0.02454369260617026*YIIIIY)]
2-iteration optimization applied!
optimization result: 1912 -> 1909
🎉 Succe

In [10]:
import numpy as np
from itertools import permutations
seq_c = [
]
seq_a = [
    PauliBit("ZI", np.pi / 4),
    PauliBit("YI", np.pi / 4),
    PauliBit("YZ", np.pi / 4),
    PauliBit("XI", np.pi / 4),
]
seq_b = [
    PauliBit("ZZ", np.pi / 4),
    PauliBit("YI", np.pi / 4),
    PauliBit("YZ", np.pi / 4),
    PauliBit("ZI", np.pi / 4),
]  
pauli_bit_equivalence_check(seq_c + seq_a, seq_b)
# for elem in permutations(seq_a):
#     if pauli_bit_equivalence_check(seq_c + list(elem), seq_b):
#         # print("Equivalent!")
#         for e in elem:
#             print(e.get_pauli_str(), np.round(e.get_angle(),2))
#         print('----'*10)

False

In [13]:
tmp = grouping(seq_a)
res = find_nontrivial_swap(tmp[0], tmp[1], tmp[2])
if res:
    pauli_bit_equivalence_check(seq_a, sum(res, []))
else:
    print(res)

None


In [ ]:
seq_a = [
    PauliBit("ZI", -np.pi / 2),
    PauliBit("XZ", -np.pi / 2),
    PauliBit("YZ", np.pi / 2),
    PauliBit("XZ", np.pi / 2),
]

-1.0

In [46]:
from qulacs.circuit import QuantumCircuitOptimizer as QCO

circuit_output = QuantumCircuit(2)
for elem in seq_a:
    circuit_output.merge_circuit(elem.convert_into_qulacs())
circuit_in = circuit_output.copy()
QCO().optimize(circuit_in, 2)
mat = circuit_in.get_gate(0).get_matrix()
print(f"Input circuit matrix: {mat}")

Input circuit matrix: [[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]


In [8]:
nqubits = 3
circuit_input = QuantumCircuit(nqubits)
for elem in data:
    circuit_input.merge_circuit(elem.convert_into_qulacs())

circuit_output = QuantumCircuit(nqubits)
circuit_output = set_clifford_to_qulacs(circuit_output, clifford_lst)
for elem in optimized_data:
    circuit_output.merge_circuit(elem.convert_into_qulacs())

equivalence_check_via_mqt_qcec(circuit_input, circuit_output, exclude_zx_checker=True)

RuntimeError: Error: QuatnumCircuit::add_gate(QuantumGateBase*): gate must be applied to qubits of which the indices are smaller than qubit_count

In [ ]:
mcr_swap(grouping(optimized_data))

MCR swap!: 1, 2 -> IIY, YXY, IIX, YXX
left_data: [PauliBit(-0.7853981633974483*IIY), PauliBit(0.7853981633974483*YXY)]
right_data: [PauliBit(-0.7853981633974483*IIX), PauliBit(-0.7853981633974483*YXX)]
MCR swap!: 3, 4 -> IIY, YXY, IIX, YXX
left_data: [PauliBit(0.7853981633974483*IIY), PauliBit(-0.7853981633974483*YXY)]
right_data: [PauliBit(0.7853981633974483*IIX), PauliBit(0.7853981633974483*YXX)]


[PauliBit(0.7853981633974483*ZZZ),
 PauliBit(-0.7853981633974483*IIX),
 PauliBit(-0.7853981633974483*YXX),
 PauliBit(-0.7853981633974483*IIY),
 PauliBit(0.7853981633974483*YXY),
 PauliBit(0.7853981633974483*IIX),
 PauliBit(0.7853981633974483*YXX),
 PauliBit(0.7853981633974483*IIY),
 PauliBit(-0.7853981633974483*YXY)]

In [ ]:
grouping(optimized_data)


[[PauliBit(0.7853981633974483*ZZZ)],
 [PauliBit(-0.7853981633974483*IIY), PauliBit(0.7853981633974483*YXY)],
 [PauliBit(-0.7853981633974483*IIX), PauliBit(-0.7853981633974483*YXX)],
 [PauliBit(0.7853981633974483*IIY), PauliBit(-0.7853981633974483*YXY)],
 [PauliBit(0.7853981633974483*IIX), PauliBit(0.7853981633974483*YXX)]]

In [7]:
import numpy as np
from mcr.gate_apply import PauliBit
from mcr.gate_apply import zhang_optimization, grouping, loop_optimization, set_clifford_to_qulacs
from mcr.mcr_optimize import find_mcr, find_nontrivial_swap
from mcr.equiv_check import equiv
seq_a = [
    PauliBit("XI", np.pi / 4),
    PauliBit("YI", -np.pi / 4),
    PauliBit("YZ", -np.pi / 4),
    PauliBit("ZI", -np.pi / 4),
    PauliBit("ZZ", np.pi / 4),
    PauliBit("XI", np.pi / 4),
    PauliBit("XZ", -np.pi / 4),
    PauliBit("ZI", -np.pi / 4),
]
m = grouping(seq_a)
m

[[PauliBit(0.7853981633974483*XI)],
 [PauliBit(-0.7853981633974483*YI), PauliBit(-0.7853981633974483*YZ)],
 [PauliBit(-0.7853981633974483*ZI), PauliBit(0.7853981633974483*ZZ)],
 [PauliBit(0.7853981633974483*XI), PauliBit(-0.7853981633974483*XZ)],
 [PauliBit(-0.7853981633974483*ZI)]]

In [9]:
find_mcr(m[2], m[3])


[]

In [10]:
{1,2,3,4,6,8,9,10}

{1, 2, 3, 4, 6, 8, 9, 10}